### MFCC 

In [ ]:
# importing required libraries
import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

bin_data_ti = pd.read_csv(r"F:\Datasets\Network_intrusion/network전처리.csv", index_col=0)

In [ ]:
# 공격 유형별로 데이터의 개수차이가 커서 같은 값으로 설정해줌 
def sampling_func(data, n_sample, seed):
    np.random.seed(seed) #실행할 때마다 동일한 샘플을 추출하기 위해 random seed 고정
    N = len(data)
    sample = data.take(np.random.permutation(N)[:n_sample])
    return sample

# valid dataset
valid_df = bin_data_ti.groupby('label', group_keys=False).apply(sampling_func, n_sample=1000, seed=777)
valid_df = valid_df.sort_index()

bin_data_ti_1=bin_data_ti.drop(valid_df.index, axis=0)

test_df = bin_data_ti_1.groupby('label', group_keys=False).apply(sampling_func, n_sample=1000, seed=7777)
test_df = test_df.sort_index()

train_df = bin_data_ti_1.drop(test_df.index, axis=0)

train_df = train_df.groupby('label', group_keys=False).apply(sampling_func, n_sample=10000, seed=1004).reset_index(drop=True)

## reset index 진행
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print(train_df['label'].value_counts())
print(valid_df['label'].value_counts())
print(test_df['label'].value_counts())

zero_label_rows = train_df[train_df['label'] == 0] # 정상데이터 
one_label_rows = train_df[train_df['label'] == 1] # 비정상 데이터 
two_label_rows = train_df[train_df['label'] == 2] # 비정상 데이터 
three_label_rows = train_df[train_df['label'] == 3] # 비정상 데이터 
four_label_rows = train_df[train_df['label'] == 4] # 비정상 데이터 
five_label_rows = train_df[train_df['label'] == 5] # 비정상 데이터 
six_label_rows = train_df[train_df['label'] == 6] # 비정상 데이터 
sev_label_rows = train_df[train_df['label'] == 7] # 비정상 데이터 
eig_label_rows = train_df[train_df['label'] == 8] # 비정상 데이터 

In [ ]:
# 각 행에 대해 MFCC 추출 및 새로운 데이터프레임에 저장
import tqdm 
import random
import pandas as pd
import numpy as np
import os
import librosa
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import tqdm 

mfcc_df = pd.DataFrame()
zero_train_label_rows = four_label_rows.iloc[:,:-1]

for index, row in tqdm.tqdm(zero_train_label_rows.iterrows()):
    # 현재 행의 오디오 시그널
    audio_signal = row.values
    
    # MFCC 추출 (여기에서는 13개의 계수만 추출)
    mfccs = librosa.feature.mfcc(y=audio_signal, sr=22050, n_mfcc=13)
    
    # 최적화된 코드
    new_columns = {f'row_{index}_mfcc{i}': mfccs[i-1] for i in range(1, 14)}
    mfcc_df = pd.concat([mfcc_df, pd.DataFrame(new_columns)], axis=1)


reshaped_array = mfcc_df.to_numpy().reshape((len(zero_train_label_rows), 13))
reshaped_train_normal = pd.DataFrame(reshaped_array)
reshaped_train_normal.to_csv('iot20d_reshaped_train_sample_4.csv', index=False)

# 결과 출력
print(reshaped_train_normal)

### PCA 적용 

In [1]:
import pandas as pd

train_normal = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_train_sample_normal.csv')
train_one = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_train_sample_1.csv')
train_two = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_train_sample_2.csv')
train_three = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_train_sample_3.csv')
train_four = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_train_sample_4.csv')
train_five = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_train_sample_5.csv')
train_six = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_train_sample_6.csv')
train_seven = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_train_sample_7.csv')
train_eight = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_train_sample_8.csv')

test_normal = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_test_sample_normal.csv')
test_one = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_test_sample_1.csv')
test_two = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_test_sample_2.csv')
test_three = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_test_sample_3.csv')
test_four = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_test_sample_4.csv')
test_five = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_test_sample_5.csv')
test_six = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_test_sample_6.csv')
test_seven = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_test_sample_7.csv')
test_eight = pd.read_csv('./IoTID20_mff전달/iot20d_reshaped_test_sample_8.csv')

In [ ]:
train_normal['label']=0
train_one['label']=1
train_two['label']=2
train_three['label']=3
train_four['label']=4
train_five['label']=5
train_six['label']=6
train_seven['label']=7
train_eight['label']=8

test_normal['label']=0
test_one['label']=1
test_two['label']=2
test_three['label']=3
test_four['label']=4
test_five['label']=5
test_six['label']=6
test_seven['label']=7
test_eight['label']=8


In [ ]:
train_merge = pd.concat([train_normal, train_one, train_two, train_three, train_four, train_five, train_six, train_seven, train_eight], ignore_index=True)
test_merge = pd.concat([test_normal, test_one, test_two, test_three, test_four, test_five, test_six, test_seven, test_eight], ignore_index=True)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(train_merge.iloc[:, :-1].values)
X_test_normalized = scaler.transform(test_merge.iloc[:, :-1].values)

n_components_pca = 12  # 주성분 개수
pca = PCA(n_components=n_components_pca)

X_train_pca = pca.fit_transform(X_train_normalized)
X_test_pca = pca.transform(X_test_normalized)

train_merge_pca = pd.DataFrame(data=np.hstack((X_train_pca, train_merge['label'].values.reshape(-1, 1))),
                               columns=[f'pca_{i}' for i in range(n_components_pca)] + ['label'])
test_merge_pca = pd.DataFrame(data=np.hstack((X_test_pca, test_merge['label'].values.reshape(-1, 1))),
                              columns=[f'pca_{i}' for i in range(n_components_pca)] + ['label'])